In [ ]:
%cd /Users/idawatibustan/Dev/shopee_data_science
%env PROJECT_PATH /Users/idawatibustan/Dev/shopee_data_science

In [ ]:
from datetime import datetime
import os

import pandas as pd

from model.heuristic.utilities import *
from model.heuristic.beauty_heuristic import run_beauty_heuristic
from utils.envs import *


In [ ]:
beauty_val = pd.read_csv(beauty_val_repo)
beauty_val.head()

In [ ]:
beauty_library = load_library('model/heuristic/beauty_library_20190322.json')

beauty_profile = beauty_library.get('primary')
beauty_profile_secondary = beauty_library.get('secondary')
beauty_profile_third = beauty_library.get('third')
beauty_max_length = beauty_library.get('length')

In [ ]:
# run heuristic function + validate accuracy on training data

beauty_train = pd.read_csv(beauty_train_repo)

start_time = datetime.now()
beauty_pred = run_beauty_heuristic(
    beauty_train, 
    beauty_profile,
    beauty_profile_secondary,
    beauty_profile_third,
    beauty_max_length
)
duration = datetime.now() - start_time
print("duration:", duration)

# accuracy on training data
for feature in beauty_profile.keys():
    print(feature)
    """ compare actual brand and prediction on insight DF"""
    insight = beauty_train.filter(['itemid', 'title', feature])
    insight[feature+'_pred'] = beauty_pred[feature]
    total = len(insight)

    """ drop brand NA of actual brand"""
    insight_accuracy = insight.dropna(subset=[feature, feature+'_pred'])
    correct_ans = insight_accuracy[feature].eq(insight_accuracy[feature+'_pred']).sum()
    total_ans = insight_accuracy[feature].count()
    print(" accuracy =", correct_ans/ total_ans)


In [ ]:
# run heuristic function on submission
start_time = datetime.now()

beauty_val_pred = run_beauty_heuristic(
    beauty_val, 
    beauty_profile,
    beauty_profile_secondary,
    beauty_profile_third,
    beauty_max_length
)

duration = datetime.now() - start_time
print(duration)

beauty_val_pred.head()

In [ ]:
# format answer for submission
beauty_val_submission = pd.DataFrame(columns=['id', 'tagging'])

for feature in beauty_profile.keys():
    temp = pd.DataFrame()
    temp['id'] = beauty_val_pred['itemid'].map(str)+"_"+feature
    temp['tagging'] = beauty_val_pred[feature]
    
    if len(beauty_val_submission) == 0:
        beauty_val_submission = temp
    else:
        beauty_val_submission = beauty_val_submission.append(temp, ignore_index=True)
    print(feature, len(beauty_val_submission))

beauty_val_submission.info()

In [ ]:
not_nan = beauty_val_submission.tagging.count()
total = len(beauty_val_submission)
print("answered = {} out of {} ({})". format(not_nan, total, not_nan/ total))

beauty_val_submission.head()

In [ ]:
outpath = 'output/result'
filename = 'beauty_info_val_submission_'+datetime.now().strftime("%Y%m%d_%H%M%S")+'.csv'
print(os.path.join(outpath,filename))

beauty_val_submission.to_csv(os.path.join(outpath,filename))